This notebook shows a simple example of running a remote function on the Ray cluster.

In [ ]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
cluster_name = "test"
namespace = "distributed"

In [ ]:
# Create our cluster and submit
# NOTE: If running outside of RHOAI notebooks, add the following line to the cluster configuration:
# namespace="rhods-notebooks"
cluster = Cluster(ClusterConfiguration(name=cluster_name, 
                                       namespace=namespace, # Update to your namespace
                                       num_workers=2, 
                                       worker_cpu_requests=4, 
                                       worker_cpu_limits=4, 
                                       worker_memory_requests=8, 
                                       worker_memory_limits=8, 
                                       head_extended_resource_requests={'nvidia.com/gpu':0},
                                       worker_extended_resource_requests={'nvidia.com/gpu':0},
                                       image="quay.io/modh/ray:2.35.0-py311-cu121",
                                       write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
                                       local_queue="local-queue"
                                       ))

In [ ]:
cluster.apply()
cluster.wait_ready()

In [ ]:
cluster.details()

In [ ]:
ray_cluster_uri = cluster.cluster_uri()
print("ray_cluster_uri: ", ray_cluster_uri)

In [ ]:
from codeflare_sdk import generate_cert

# Create required TLS cert and export the environment variables to enable TLS
generate_cert.generate_tls_cert(cluster_name, namespace)
generate_cert.export_env(cluster_name, namespace)

In [ ]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray

# reset ray context in case there's already one
ray.shutdown()

In [ ]:
runtime_env = {"pip": ["emoji"]}

ray.init(address=ray_cluster_uri, runtime_env=runtime_env)
print("Ray cluster is up and running ", ray.is_initialized())

In [ ]:
@ray.remote
def f():
  import emoji
  return emoji.emojize('Python is :thumbs_up:')

print(ray.get(f.remote()))

In [ ]:
cluster.down()